In [22]:
# import numpy as np
from pprint import pprint
from pathlib import Path


In [23]:
fname = Path(r'input.txt')
fname_test = Path(r'test.txt')

with open(fname, 'r') as fp:
    lines = fp.readlines()

with open(fname_test, 'r') as fp:
    lines_test = fp.readlines()


In [24]:
class Hailstone():
    def __init__(self, x, y, z, vx, vy, vz):
        self.x0 = int(x)
        self.y0 = int(y)
        self.z0 = int(z)
        self.vx = int(vx)
        self.vy = int(vy)
        self.vz = int(vz)


    def __repr__(self):
        return f'Hailstone(x0={self.x0}, y0={self.y0}, z0={self.z0}, vx={self.vx}, vy={self.vy}, vz={self.vz})'
    

    def get_t(self, *, x=None, y=None):
        """Get t from either x or y"""
        if x is not None:
            t = (x - self.x0) / self.vx
        else:
            t = (y - self.y0) / self.vy
        return t
    
    @property
    def a(self):
        return np.array([[-self.vy, self.vx]], np.float_, ndmin=2)
    
    @property
    def b(self):
        return np.array([self.vx * self.y0 - self.vy * self.x0], np.float_, ndmin=1)
    

    def collision_position(self, other):
        """Return x, y position of collision"""
        a_mat = np.concatenate([self.a, other.a])
        b_mat = np.concatenate([self.b, other.b])
        result = np.dot(np.linalg.inv(a_mat), b_mat)
        return result
    
    def test_collision(self, other):
        # print(f'{self}, {other}')
        try:
            result = self.collision_position(other)
            # print(result)
        except np.linalg.LinAlgError as e:
            # Singluar matrix: no solution, parallel lines
            # print(" no match")
            return False
        if (result < XMIN).any():
            # print(' too low')
            return False
        if (result > XMAX).any():
            # print(' too high')
            return False
        if self.get_t(x=result[0]) < 0: 
            # print(' self in the past')
            return False
        if other.get_t(x=result[0]) < 0:
            # print(' other in the past')
            return False
        
        # print(' added')
        return True


In [25]:
hail_list = []
for ln in lines:
    pos_vel = ln.strip().replace('@', ',').split(r',')
    hail_list.append(Hailstone(*pos_vel))

hail_list_test = []
for ln in lines_test:
    pos_vel = ln.strip().replace('@', ',').split(r',')
    hail_list_test.append(Hailstone(*pos_vel))

pprint(hail_list_test)


[Hailstone(x0=19, y0=13, z0=30, vx=-2, vy=1, vz=-2),
 Hailstone(x0=18, y0=19, z0=22, vx=-1, vy=-1, vz=-2),
 Hailstone(x0=20, y0=25, z0=34, vx=-2, vy=-2, vz=-4),
 Hailstone(x0=12, y0=31, z0=28, vx=-1, vy=-2, vz=-1),
 Hailstone(x0=20, y0=19, z0=15, vx=1, vy=-5, vz=-3)]


In [26]:
def collision_time(hailstone, other_object):
    dx0 = hailstone.x0 - other_object.x0
    dy0 = hailstone.y0 - other_object.y0
    dz0 = hailstone.z0 - other_object.z0
    dvx = hailstone.vx - other_object.vx
    dvy = hailstone.vy - other_object.vy
    dvz = hailstone.vz - other_object.vz

    t = -((dx0*dvx + dy0*dvy + dz0*dvz) / (dvx**2 + dvy**2 + dvz**2))
    return t


In [27]:
# Test--check collision times correct--model brick as a Hailstone

for hail_stone in hail_list_test:
    print(collision_time(hail_stone, Hailstone(24,13,10,-3,1,2)))

5.0
3.0
4.0
6.0
1.0


In [28]:
def min_seperation_squared(hailstone, other_object):
    dx0 = hailstone.x0 - other_object.x0
    dy0 = hailstone.y0 - other_object.y0
    dz0 = hailstone.z0 - other_object.z0
    dvx = hailstone.vx - other_object.vx
    dvy = hailstone.vy - other_object.vy
    dvz = hailstone.vz - other_object.vz

    t = -((dx0*dvx + dy0*dvy + dz0*dvz) / (dvx**2 + dvy**2 + dvz**2))

    dx2 = (dx0 + dvx*t)**2
    dy2 = (dy0 + dvy*t)**2
    dz2 = (dz0 + dvz*t)**2

    r2 = dx2 + dy2 + dz2
    return r2

In [29]:
# Test
for hail_stone in hail_list_test:
    print(min_seperation_squared(hail_stone, Hailstone(24,13,10,-3,1,2)))

0.0
0.0
0.0
0.0
0.0


In [30]:
def display_two_hailstones(h1_indx, h2_indx, h1, h2):
    print(f'{h1_indx:<4}   {h1}\n->{h2_indx:<3}  {h2}')


In [31]:
for h1_indx in range(len(hail_list) - 1):
    for h2_indx in range(h1_indx+1, len(hail_list)):
        h1 = hail_list[h1_indx]
        h2 = hail_list[h2_indx]
        if h1.x0 == h2.x0 and h1.vx == h2.vx:
            display_two_hailstones(h1_indx, h2_indx, h1, h2)
        if h1.y0 == h2.y0 and h1.vy == h2.vy:
            display_two_hailstones(h1_indx, h2_indx, h1, h2)
        if h1.z0 == h2.z0 and h1.vz == h2.vz:
            display_two_hailstones(h1_indx, h2_indx, h1, h2)


239    Hailstone(x0=300193693260508, y0=151611923410873, z0=231363386790708, vx=185, vy=-134, vz=71)
->268  Hailstone(x0=306960548670948, y0=230204613282241, z0=231363386790708, vx=-38, vy=-112, vz=71)


Hailstones 239 and 268 have the same z coordinate and z velocity. Therefore the brick must pass through the point they collide in the x-y plane OR must have vz == 0. Since none of the hailstones will collide then vz must be zero.

In [32]:
hs1 = hail_list[239]
hs2 = hail_list[268]



# Update rock parameters
We now know z0 and vz for the rock--must be in the same x-y plane as hs1 and hs2

In [33]:
rock = {'z0': hs1.z0, 
        'vz': hs1.vz}

Move into hailstone 239's frame of reference.

In [34]:
hs0 = hs1
transformed_hail_list = [
    Hailstone(hs.x0 - hs0.x0, 
              hs.y0 - hs0.y0, 
              hs.z0 - hs0.z0, 
              hs.vx - hs0.vx, 
              hs.vy - hs0.vy, 
              hs.vz - hs0.vz)
        for hs in hail_list]
transformed_hail_list[:10]

# suffix 'p' (==primed) -- value in hs1 reference frame (RF1). Let RF0 == rest reference frame

hs1p = transformed_hail_list[239]
hs2p = transformed_hail_list[268]
rock['z0p'] = hs1p.z0
rock['vzp'] = hs1p.vz

rock

{'z0': 231363386790708, 'vz': 71, 'z0p': 0, 'vzp': 0}

In [35]:
# Check for other hailstones with two components of velocity the same

for indx, h1 in enumerate(transformed_hail_list):
    if h1.vx == h1.vy or h1.vy == h1.vz or h1.vz == h1.vx:
        print(indx, h1)


26 Hailstone(x0=-173920921494258, y0=19551577975824, z0=-215958551457157, vx=31, vy=331, vz=331)
37 Hailstone(x0=-52402971967504, y0=187762958632892, z0=15431755619736, vx=-99, vy=-99, vz=-36)
239 Hailstone(x0=0, y0=0, z0=0, vx=0, vy=0, vz=0)


# Use these new hailstones to solve equations for rock

In [36]:
hs3 = hail_list[26]  # vy == vz
hs4 = hail_list[37]  # vx == vy

hs3p = transformed_hail_list[26]
hs4p = transformed_hail_list[37]

In [37]:
# To find vxp, vyp for rock
# hs3_{t, xp, yp} are time, x, and y coord (in RF1) when rock collides with hailstone.

# When hs3 gets to RF1:
hs3_t = -hs3p.z0 / hs3p.vz

# transformed y position at this time:
hs3_yp = hs3p.y0 + hs3p.vy * hs3_t
print(hs3_t, hs3_yp)

# transformed x pos at this time:
hs3_xp = hs3p.x0 + hs3p.vx * hs3_t
print(hs3_xp)

# now know that in RF1, rock passes through (0,0,0) and (hs3_xp, hs3_yp, 0)
# so rock['vxp'] / rock['vyp'] = hs3_xp / hs3_yp

652442753647.0 235510129432981.0
-153695196131201.0


In [38]:
# repeat for HS4

# When hs4 gets to RF1:
hs4_t = -hs4p.z0 / hs4p.vz

# transformed y position at this time:
hs4_yp = hs4p.y0 + hs4p.vy * hs4_t
print(hs4_t, hs4_yp)

# transformed x pos at this time:
hs4_xp = hs4p.x0 + hs4p.vx * hs4_t
print(hs4_xp)

428659878326.0 145325630678618.0
-94840299921778.0


In [39]:
# We don't know when rock passes through (0,0,0). We do now know two points, collision with HS3 and HS4. 
# We can solve to find angle rock must move in

rock['vyp'] = (hs4_yp - hs3_yp) / (hs4_t - hs3_t)
rock['vxp'] = (hs4_xp - hs3_xp) / (hs4_t - hs3_t)

pprint(rock)
print(rock['vxp'] / rock['vyp'])
print(hs3_xp / hs3_yp)  # should be the same as previous line
print(hs4_xp / hs4_yp)  # should be the same as previous line

{'vxp': -263.0,
 'vyp': 403.0,
 'vz': 71,
 'vzp': 0,
 'z0': 231363386790708,
 'z0p': 0}
-0.652605459057072
-0.652605459057072
-0.652605459057072


In [40]:
# Get time rock passes through origin in RF1

t03 = hs3_t - (hs3_xp / rock['vxp'])
t04 = hs4_t - (hs4_xp / rock['vxp'])
print(t03)  # should be the same
print(t04)

t0 = t03

68050372920.0
68050372920.0


In [41]:
# Calculate back to get x0, y0 in RF1

rock['x0p'] = -rock['vxp'] * t0
rock['y0p'] = -rock['vyp'] * t0
rock

{'z0': 231363386790708,
 'vz': 71,
 'z0p': 0,
 'vzp': 0,
 'vyp': 403.0,
 'vxp': -263.0,
 'x0p': 17897248077960.0,
 'y0p': -27424300286760.0}

In [42]:
# Transform rooc coordinates back to rest frame RF0

rock['x0'] = rock['x0p'] + hs1.x0
rock['y0'] = rock['y0p'] + hs1.y0
print(rock)
result = sum([rock[coord] for coord in ['x0', 'y0', 'z0']])
print(int(result))

{'z0': 231363386790708, 'vz': 71, 'z0p': 0, 'vzp': 0, 'vyp': 403.0, 'vxp': -263.0, 'x0p': 17897248077960.0, 'y0p': -27424300286760.0, 'x0': 318090941338468.0, 'y0': 124187623124113.0}
673641951253289
